In [15]:
import geopandas as gpd
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from shapely import wkt

In [16]:
df = pd.read_csv('ads24-buildings.csv', low_memory=False)
df = df.head(10000)
df

,id,building,wkt
0,201782706,yes,"LINESTRING(18.9918839 50.1900409,18.9919257 50..."
1,361329065,yes,"LINESTRING(20.615881 52.8893058,20.6158402 52...."
2,361329066,yes,"LINESTRING(20.6141094 52.888029200000005,20.61..."
3,361329067,yes,"LINESTRING(20.611661700000003 52.8855821,20.61..."
4,201782707,yes,"LINESTRING(18.9976388 50.194090800000005,18.99..."
...,...,...,...
9995,35585843,apartments,"LINESTRING(16.9718252 52.3870815,16.9719474 52..."
9996,202103042,yes,"LINESTRING(18.9833137 50.266285,18.9832532 50...."
9997,51452681,transportation,"LINESTRING(14.4982105 53.438990600000004,14.49..."
9998,389173825,yes,"LINESTRING(19.852201700000002 49.6888991,19.85..."


In [19]:
df['geometry'] = df['wkt'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry='geometry')
gdf['lat'] = gdf.centroid.x
gdf['long'] = gdf.centroid.y

gdf.to_crs(epsg=2180, inplace=True)
gdf['x'] = gdf.centroid.x
gdf['y'] = gdf.centroid.y
gdf = pd.concat([gdf, gdf.bounds], axis=1)
gdf['wkt'] = gdf['geometry'].apply(wkt.dumps)
gdf['radius'] = gdf.apply(lambda row: np.sqrt((row['x'] - row['maxx'])** 2 + (row['y'] - row['maxy'])** 2), axis=1)
gdf = gdf.drop(['geometry', 'minx', 'miny', 'maxx', 'maxy'], axis=1)
gdf

/tmp/ipykernel_392702/3143812975.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.x
/tmp/ipykernel_392702/3143812975.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['long'] = gdf.centroid.y


,id,building,wkt,lat,long,x,y,radius
0,201782706,yes,LINESTRING (499420.8127744635567069 258092.102...,18.991903,50.190017,499422.207214,258089.392603,3.141233
1,361329065,yes,LINESTRING (608682.9840914446394891 559420.450...,20.615891,52.889275,608683.761959,559417.091868,4.812712
2,361329066,yes,LINESTRING (608567.0259966760640964 559275.821...,20.614128,52.887998,608568.376558,559272.338811,5.068983
3,361329067,yes,LINESTRING (608408.5076238969340920 559000.020...,20.611670,52.885547,608409.160929,558996.104336,5.531173
4,201782707,yes,LINESTRING (499831.5125229217228480 258542.239...,18.997626,50.194074,499830.569477,258540.426897,2.489833
...,...,...,...,...,...,...,...,...
9995,35585843,apartments,LINESTRING (362017.5607251165783964 504284.942...,16.971179,52.387684,361975.502990,504353.088139,83.914251
9996,202103042,yes,LINESTRING (498811.1167561059119180 266567.162...,18.983242,50.266265,498806.028310,266564.923677,7.506956
9997,51452681,transportation,LINESTRING (201135.5722427452565171 628768.388...,14.498264,53.438941,201138.669924,628762.787904,7.252645
9998,389173825,yes,LINESTRING (561449.1905924356542528 202738.627...,19.852262,49.688866,561453.568342,202734.959610,6.314716


In [13]:
x = gdf.iloc[1]['x']
y = gdf.iloc[1]['y']
dx = gdf.iloc[1]['maxx'] - gdf.iloc[1]['minx']
dy = gdf.iloc[1]['maxy'] - gdf.iloc[1]['miny']
cx = (gdf.iloc[1]['maxx'] + gdf.iloc[1]['minx']) / 2
cy = (gdf.iloc[1]['maxy'] + gdf.iloc[1]['miny']) / 2
r = np.sqrt((dx/2)**2 + (dy/2)**2)
r2 = np.sqrt((x-gdf.iloc[1]['maxx'])** 2 + (y-gdf.iloc[1]['maxy'])** 2)
x, y, cx ,cy ,dx, dy, r, r2

(608683.7619591521,
 559417.0918684973,
 608683.7628211451,
 559417.0917134532,
 6.891833551577292,
 6.71802279446274,
 4.812203236812828,
 4.812712320771185)